In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re

df = pd.read_csv("https://raw.githubusercontent.com/ekoly/DS-Unit-2-Applied-Modeling/master/data/ufc/data.csv")

In [0]:
# Define ML problems

# You will use your portfolio project dataset for all assignments this sprint.

# Assignment

# Complete these tasks for your project, and document your decisions.

# [ ] Choose your target. Which column in your tabular dataset will you predict?
# [ ] Is your problem regression or classification?
# [ ] How is your target distributed?

# Classification: How many classes? Are the classes imbalanced?

# Regression: Is the target right-skewed? If so, you may want to log transform the target.

# [ ] Choose your evaluation metric(s).

# Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. 
# Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
# Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?

# [ ] Choose which observations you will use to train, validate, and test your model.
# Are some observations outliers? Will you exclude them?
# Will you do a random split or a time-based split?

# [ ] Begin to clean and explore your data.
# [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

In [0]:
df.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,...,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019-06-08,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0.0,4.0,0.0,9.200000,6.000000,0.200000,0.000000,62.600000,20.600000,2.600000,2.000000,48.600000,11.200000,0.800000,7.6,5.400000,0.400000,0.000000,65.40,22.600000,0.466000,0.400000,0.80000,0.200000,0.100000,66.400000,23.600000,4.0,1.0,6.400000,4.000000,...,13.300000,8.800000,7.500000,5.100000,90.500000,26.800000,0.800000,0.300000,76.100000,17.300000,0.100000,9.400000,6.100000,0.000000,0.000000,98.800000,32.200000,0.336000,0.000000,0.900000,0.100000,0.050000,110.500000,43.300000,27.0,742.60,3.0,0.0,2.0,4.0,2.0,0.0,0.0,8.0,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019-06-08,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0.0,3.0,0.0,14.600000,9.100000,11.800000,7.300000,124.700000,42.100000,2.400000,1.900000,112.000000,32.000000,0.000000,12.3,10.200000,0.800000,0.000000,138.90,51.300000,0.399000,0.700000,1.00000,0.500000,0.225000,158.700000,69.600000,3.0,6.0,13.000000,9.300000,...,24.571429,14.142857,10.571429,7.857143,98.571429,32.571429,6.428571,4.285714,61.857143,12.428571,0.000000,29.142857,18.142857,1.142857,0.000000,115.571429,44.714286,0.437143,0.285714,3.285714,0.857143,0.147143,158.142857,82.285714,25.0,1062.00,2.0,0.0,1.0,2.0,0.0,2.0,0.0,5.0,Southpaw,165.10,167.64,125.0,32.0,31.0
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Red,False,Lightweight,3,0.0,3.0,0.0,15.354839,11.322581,6.741935,4.387097,84.741935,38.580645,5.516129,3.806452,67.645161,23.258065,0.645161,14.0,12.193548,0.935484,0.096774,97.00,46.774194,0.496129,0.354839,2.16129,0.677419,0.295484,103.709677,52.548387,8.0,8.0,17.903226,11.870968,...,14.466667,8.133333,2.800000,0.733333,91.066667,32.200000,4.866667,2.800000,78.266667,23.200000,0.266667,6.000000,4.400000,0.333333,0.133333,98.733333,35.733333,0.340000,0.066667,2.866667,0.666667,0.131333,102.133333,38.600000,33.0,604.40,2.0,0.0,1.0,3.0,3.0,6.0,1.0,14.0,Orthodox,180.34,193.04,155.0,36.0,35.0
3,Jimmie Rivera,Petr Yan,Kevin MacDonald,2019-06-08,"Chicago, Illinois, USA",Blue,False,Bantamweight,3,0.0,4.0,0.0,17.000000,14.000000,13.750000,11.000000,109.500000,48.750000,13.000000,10.500000,116.250000,53.750000,0.500000,3.0,2.500000,0.500000,0.250000,136.25,70.250000,0.550000,0.250000,2.50000,1.250000,0.287500,154.750000,86.750000,4.0,0.0,12.250000,6.000000,...,20.250000,13.375000,6.875000,5.625000,103.125000,38.500000,0.875000,0.750000,77.375000,20.375000,0.125000,13.250000,11.125000,0.000000,0.000000,110.875000,44.875000,0.446250,0.000000,2.375000,0.000000,0

In [0]:

r_fighter_cols = [col for col in df.columns if col.startswith("R_")]
b_fighter_cols = [col for col in df.columns if col.startswith("B_")]

other_cols = [col for col in df.columns if col not in r_fighter_cols and col not in b_fighter_cols]

def extractFighters(df, fighter_cols, opp_cols, fighter_prefix, opp_prefix, fighter_color):

    fighters_df = df[fighter_cols + other_cols].copy()
    fighters_df.columns = [col.replace(fighter_prefix, "") for col in fighters_df.columns]
    fighters_df["is_winner"] = fighters_df["Winner"] == fighter_color

    opp_df = df[opp_cols].copy()
    opp_df.columns = [re.sub(r"^" + opp_prefix, "opponent_", col) for col in opp_df.columns]

    fighters_df = pd.concat([fighters_df, opp_df], axis=1)
    
    return fighters_df

r_fighters_df = extractFighters(df, r_fighter_cols, b_fighter_cols, "R_", "B_", "Red")
b_fighters_df = extractFighters(df, b_fighter_cols, r_fighter_cols, "B_", "R_", "Blue")

fighters_df = pd.concat([r_fighters_df, b_fighters_df], axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [0]:
fighters_df['fighter'].value_counts()

Jim Miller           32
Donald Cerrone       32
Jeremy Stephens      30
Michael Bisping      29
Demian Maia          29
                     ..
Anthony Hernandez     1
Rolles Gracie         1
Ben Askren            1
Jon Manley            1
Hugo Duarte           1
Name: fighter, Length: 1915, dtype: int64

In [0]:
df.shape

(5144, 145)

In [0]:
pd.options.display.max_seq_items = None
fighters_df.columns

Index(['Height_cms', 'Reach_cms', 'Referee', 'Stance', 'Weight_lbs', 'Winner',
       'age', 'avg_BODY_att', 'avg_BODY_landed', 'avg_CLINCH_att',
       'avg_CLINCH_landed', 'avg_DISTANCE_att', 'avg_DISTANCE_landed',
       'avg_GROUND_att', 'avg_GROUND_landed', 'avg_HEAD_att',
       'avg_HEAD_landed', 'avg_KD', 'avg_LEG_att', 'avg_LEG_landed',
       'avg_PASS', 'avg_REV', 'avg_SIG_STR_att', 'avg_SIG_STR_landed',
       'avg_SIG_STR_pct', 'avg_SIG_STatt', 'avg_SIG_STlanded', 'avg_SIG_STpct',
       'avg_SUATT', 'avg_SUB_ATT', 'avg_TD_att', 'avg_TD_landed', 'avg_TD_pct',
       'avg_TOTAL_STR_att', 'avg_TOTAL_STR_landed', 'avg_TOTAL_STatt',
       'avg_TOTAL_STlanded', 'avg_opp_BODY_att', 'avg_opp_BODY_landed',
       'avg_opp_CLINCH_att', 'avg_opp_CLINCH_landed', 'avg_opp_DISTANCE_att',
       'avg_opp_DISTANCE_landed', 'avg_opp_GROUND_att',
       'avg_opp_GROUND_landed', 'avg_opp_HEAD_att', 'avg_opp_HEAD_landed',
       'avg_opp_KD', 'avg_opp_LEG_att', 'avg_opp_LEG_landed', 'avg_op

In [0]:
# [ ] Choose your target. Which column in your tabular dataset will you predict?
#. Going to predict whether a fighter wins or not.
# [ ] Is your problem regression or classification?
#. The problem is classification (binary)
# [ ] How is your target distributed?
#. 51% are losses, 49% are wins
# Classification: How many classes? Are the classes imbalanced?
#. 2 classes, they are very balanced
# Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. 
#. Yes it is - Using accuracy.


fighters_df['is_winner'].value_counts(normalize=True)

False    0.508068
True     0.491932
Name: is_winner, dtype: float64

In [0]:
# [ ] Choose which observations you will use to train, validate, and test your model.
# Are some observations outliers? Will you exclude them?
# Will you do a random split or a time-based split?

In [0]:
for col in [col for col in fighters_df.select_dtypes(include="number").columns if col.startswith("opponent_")]:
    
    col2 = col
    col1 = col[9:]
    
    fighters_df[col1 + "_ratio"] = (fighters_df[col1] + 1) / (fighters_df[col2] + 1)
    
# prevent leakage
fighters_df = fighters_df.drop(columns="Winner")

# change prefixes to suffixes
fighters_df.columns = [col[9:] + "_opponent" if col.startswith("opponent_") else col for col in fighters_df.columns]

In [0]:
fighters_df.head()

,Height_cms,Reach_cms,Referee,Stance,Weight_lbs,age,avg_BODY_att,avg_BODY_landed,avg_CLINCH_att,avg_CLINCH_landed,avg_DISTANCE_att,avg_DISTANCE_landed,avg_GROUND_att,avg_GROUND_landed,avg_HEAD_att,avg_HEAD_landed,avg_KD,avg_LEG_att,avg_LEG_landed,avg_PASS,avg_REV,avg_SIG_STR_att,avg_SIG_STR_landed,avg_SIG_STR_pct,avg_SIG_STatt,avg_SIG_STlanded,avg_SIG_STpct,avg_SUATT,avg_SUB_ATT,avg_TD_att,avg_TD_landed,avg_TD_pct,avg_TOTAL_STR_att,avg_TOTAL_STR_landed,avg_TOTAL_STatt,avg_TOTAL_STlanded,avg_opp_BODY_att,avg_opp_BODY_landed,avg_opp_CLINCH_att,avg_opp_CLINCH_landed,...,avg_TOTAL_STR_landed_ratio,avg_opp_BODY_att_ratio,avg_opp_BODY_landed_ratio,avg_opp_CLINCH_att_ratio,avg_opp_CLINCH_landed_ratio,avg_opp_DISTANCE_att_ratio,avg_opp_DISTANCE_landed_ratio,avg_opp_GROUND_att_ratio,avg_opp_GROUND_landed_ratio,avg_opp_HEAD_att_ratio,avg_opp_HEAD_landed_ratio,avg_opp_KD_ratio,avg_opp_LEG_att_ratio,avg_opp_LEG_landed_ratio,avg_opp_PASS_ratio,avg_opp_REV_ratio,avg_opp_SIG_STR_att_ratio,avg_opp_SIG_STR_landed_ratio,avg_opp_SIG_STR_pct_ratio,avg_opp_SUB_ATT_ratio,avg_opp_TD_att_ratio,avg_opp_TD_landed_ratio,avg_opp_TD_pct_ratio,avg_opp_TOTAL_STR_att_ratio,avg_opp_TOTAL_STR_landed_ratio,current_lose_streak_ratio,current_win_streak_ratio,draw_ratio,longest_win_streak_ratio,losses_ratio,total_rounds_fought_ratio,total_time_fought(seconds)_ratio,total_title_bouts_ratio,win_by_Decision_Majority_ratio,win_by_Decision_Split_ratio,win_by_Decision_Unanimous_ratio,win_by_KO/TKO_ratio,win_by_Submission_ratio,win_by_TKO_Doctor_Stoppage_ratio,wins_ratio
0,162.56,162.56,Marc Goddard,Orthodox,135.0,32.0,21.900000,16.400000,17.000000,11.000000,75.000000,26.500000,9.400000,6.500000,74.200000,23.90,0.400,5.300000,3.700000,1.200000,0.000000,NaN,NaN,NaN,101.400000,44.000000,0.466000,NaN,0.100000,5.300000,1.900000,0.458000,NaN,NaN,129.900000,69.100000,13.300000,8.800000,7.500000,5.100000,...,NaN,1.932432,1.960000,4.250000,3.812500,1.752874,1.510870,1.125000,1.083333,1.899015,1.759615,0.916667,1.333333,1.224138,1.000000,1.000000,NaN,NaN,NaN,1.000000,0.950000,0.785714,0.954545,NaN,NaN,1.0,1.00,1.0,1.000000,1.500000,2.800000,1.768792,4.0,1.0,1.500000,5.000000,1.000000,0.5,1.0,1.800
1,165.10,167.64,Robert Madrigal,Southpaw,125.0,31.0,12.000000,7.714286,9.285714,6.857143,88.142857,36.142857,18.428571,16.428571,84.571429,37.00,0.000,19.285714,14.714286,1.714286,0.142857,NaN,NaN,NaN,115.857143,59.428571,0.575714,NaN,0.428571,5.142857,2.428571,0.601429,NaN,NaN,161.571429,102.857143,24.571429,14.142857,10.571429,7.857143,...,NaN,1.826531,1.470180,0.838509,0.835580,0.969537,1.017316,0.816327,0.669078,0.636850,0.422282,0.909091,2.336656,2.036474,0.892857,1.000000,NaN,NaN,NaN,0.756303,1.298701,0.977444,0.931879,NaN,NaN,1.0,0.75,1.0,0.750000,0.428571,0.866667,1.250588,3.0,1.0,0.666667,1.500000,1.000000,3.0,0.5,1.200
2,180.34,193.04,Dan Miragliotta,Orthodox,155.0,35.0,13.866667,8.666667,2.866667,1.733333,116.133333,49.466667,5.333333,4.266667,96.733333,35.60,0.200,13.733333,11.200000,0.333333,0.133333,NaN,NaN,NaN,124.333333,55.466667,0.430000,NaN,1.000000,0.933333,0.400000,0.277333,NaN,NaN,133.000000,63.400000,14.466667,8.133333,2.800000,0.733333,...,NaN,0.818203,0.709607,0.403425,0.253459,1.076194,0.849876,2.139608,1.963333,1.154731,0.913764,1.033333,0.688889,0.636502,1.291667,1.097917,NaN,NaN,NaN,0.972549,1.248611,1.359649,1.063735,NaN,NaN,1.0,3.00,1.0,1.333333,0.222222,0.492754,1.038652,1.5,1.0,2.000000,0.500000,0.363636,1.0,2.0,0.625
3,162.56,172.72,Kevin MacDonald,Orthodox,135.0,29.0,18.250000,10.250000,5.875000,4.125000,104.875000,41.000000,1.000000,0.625000,80.500000,24.00,0.375,13.000000,11.500000,0.125000,0.000000,NaN,NaN,NaN,111.750000,45.750000,0.366250,NaN,0.000000,2.250000,0.625000,0.103750,NaN,NaN,117.375000,50.750000,20.250000,13.375000,6.875000,5.625000,...,NaN,1.603774,2.053571,1.125000,1.394737,1.093176,1.423423,0.681818,0.777778,0.938623,0.950000,0.900000,1.727273,2.309524,1.000000,1.000000,NaN,NaN,NaN,1.000000,0.613636,0.571429,0.911162,NaN,NaN,2.0,0.20,1